In [38]:
import duckdb
from lonboard import PolygonLayer, Map, PathLayer

import matplotlib as mpl
from matplotlib.colors import LogNorm

import pandas as df



db = duckdb.connect()
 
db.execute("INSTALL spatial;")
db.execute("LOAD spatial;")
# This line will load the optimized parquet file
# db.execute("CREATE TABLE ssa_times AS FROM '../data/optimized.parquet';")
# This line will load the parquet file for the DOGE selected offices
db.execute("CREATE TABLE ssa_times AS FROM '../data/all_ssa_times.parquet';")
sql = """ SELECT 
        ST_ASWKB(ssa_times.geom) as geom, 
        P.over_65, 
        P.total , 
        destination_id as d, 
        nearest_office, 
        increase_sec, 
        now_duration_sec, 
        after_close_duration_sec, 
        after_close_office, 
        coalesce(per_gop, 0) per_gop,
        coalesce(per_dem, 0) per_dem
    FROM ssa_times 
    JOIN './data/tract/cb_2023_us_tract_5m.shp' AS G ON G.GEOID = destination_id
    LEFT OUTER JOIN './data/ssa-populations.csv' P ON 
        destination_id = P.geoid
    LEFT OUTER JOIN './data/2024_US_County_Level_Presidential_Results.csv' ON
        destination_id LIKE concat(county_fips, '%');""" 
query = db.sql(sql)



res = query.execute().df()




colormap = mpl.colormaps["plasma"]
color_normalizer = mpl.colors.Normalize(1, vmax=res['increase_sec'].max()) #Normalize population to the 0-1 range
# color_normalizer = mpl.colors.Normalize(0, vmax=1) #Normalize population to the 0-1 range
# color_normalizer = mpl.colors.Normalize(1, vmax=4 * 60 * 60) #Normalize population to the 0-1 range
colors = colormap(color_normalizer(res['increase_sec']), bytes=True)
# colors[:,3] = 127

height_normalizer = mpl.colors.Normalize(0, vmax=res['total'].max())
heights = height_normalizer(res['total']) * 400000


layer = PolygonLayer.from_duckdb(
    query,
    # auto_highlight=True,
    get_line_color="black",
    get_line_width=2,
    get_fill_color=colors,
    get_elevation=heights,
    # extruded=True,
    # opacity=0.02,
    highlight_color=[255,0,0]
)

m = Map(layer)
m._height = 1024
m.to_html('current_trip_times.html')
m


FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

/Users/schreiaj/project/sys-gov-demos/ssa-office-closure/venv/lib/python3.11/site-packages/lonboard/_geoarrow/ops/reproject.py:33: UserWarning: No CRS exists on data. If no data is shown on the map, double check that your CRS is WGS84.
  warn(


Map(custom_attribution='', layers=(PolygonLayer(get_elevation=arro3.core.ChunkedArray<Float32>
, get_fill_colo…